# Preprocessing - Polygenic Risk Scores

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import lifelines

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import neptune
import warnings
warnings.filterwarnings("ignore")
import shutil
import pathlib

dataset_name = "210212_cvd_gp"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data"
dataset_path = f"{data_path}/2_datasets_pre/{dataset_name}"

In [ ]:
data = pd.read_feather(f"{dataset_path}/baseline_covariates.feather")
data.head()

## Score Weights

PGS Catalogue searched for ["Atrial Fibrillation", "Coronary Artery Disease", "Coronary Heart Disease", "Stroke"] 

In [ ]:
pgs_planned = sorted([p.absolute() for p in list(pathlib.Path(f"{data_path}/1_genetics/pgs_weights/raw").rglob('*.txt'))])
pgs_planned_list = [p.name[:-4] for p in pgs_planned]
pgs_planned_dict = dict(zip(pgs_planned_list, pgs_planned))

## Get Scores

In [ ]:
pgs_finished = sorted([p.parent.absolute() for p in list(pathlib.Path(f"{data_path}/1_genetics").rglob('*distribution_plot.png'))])
pgs_list = [p.name for p in pgs_finished]
pgs_dict = dict(zip(pgs_list, pgs_finished))

In [ ]:
data_pgs = data[["eid"]]
for pgs, pgs_path in pgs_dict.items():
    temp = pd.read_csv(str(pgs_path)+"/PRSice.all_score", delim_whitespace=True)
    temp = temp[temp.columns[-2:]]
    temp.columns = ["eid", pgs]
    data_pgs = data_pgs.merge(temp, on="eid", how="left")

In [ ]:
for pgs in pgs_dict.keys():
    data_pgs[pgs].plot.kde()

## Plotting

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import DataFrameMapper
        
leave_eid = [(col, None) for col in ["eid"]]
standardize = [([col], StandardScaler()) for col in list(pgs_dict.keys())]

mapper = DataFrameMapper(leave_eid + standardize, df_out=True)
data_pgs_tf = mapper.fit_transform(data_pgs)

In [ ]:
for pgs in pgs_dict.keys():
    data_pgs_tf[pgs].plot.kde()

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt 

def corrfunc(x,y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    # Unicode for lowercase rho (ρ)
    rho = '\u03C1'
    ax.annotate(f'{rho} = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
 
# Basic correlogram
g = sns.pairplot(data_pgs_tf[list(pgs_dict.keys())])
g.map_lower(corrfunc)
plt.show()

## Write to disk

In [ ]:
dtypes = {"int32":"int", "int64":"int", "float64":"float", "category":"category", "object":"category", "bool":"bool"}
desc_dict = {"id": [*range(1, len(data_pgs.columns.to_list())+1)] , 
             "covariate": data_pgs.columns.to_list(), 
             "dtype":[dtypes[str(col)] for col in data_pgs.dtypes.to_list()], 
             "isTarget":[False for col in data_pgs.columns.to_list()],
            "based_on":["PGS" if col!="eid" else "eid" for col in data_pgs.columns.to_list()],
            "aggr_fn": [np.nan for col in data_pgs.columns.to_list()]}
data_pgs_description = pd.DataFrame.from_dict(desc_dict)
data_pgs_description

In [ ]:
data_pgs.to_feather(f"{dataset_path}/baseline_pgs.feather")
data_pgs_description.to_feather(f"{dataset_path}/baseline_pgs_description.feather")